# Docx-Exporter 실행코드

In [13]:
from docx import Document
import pandas as pd
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
from tkinter import * # __all__
from tkinter import filedialog
import threading, time

OP_ROW = 1 
HEADER_ROW = 2 
FIRST_ROW = 3 

# Functions

In [14]:
def getLeftHeaderCols(table):
    op_cols = []
    ref_cols = []  
    item_cols = []
    des_cols = []
    qty_cols = []
    for idx, _ in enumerate(table.columns):
        if 'op' in table.cell(OP_ROW, idx).text.lower().strip():
            op_cols.append(idx)
        if 'ref' in table.cell(HEADER_ROW, idx).text.lower().strip():
            ref_cols.append(idx)
        if 'man.item' in table.cell(HEADER_ROW, idx).text.lower().strip():
            item_cols.append(idx)
        if 'description' in table.cell(HEADER_ROW, idx).text.lower().strip():
            des_cols.append(idx)
        if 'qty' in table.cell(HEADER_ROW, idx).text.lower().strip():
            qty_cols.append(idx)

    try:        
        return {'op':op_cols[0] ,'ref':ref_cols[0],'item':item_cols[0],'des':des_cols[0],'qty':qty_cols[0]}
    except:
        print('out of range')

def getRightHeaderCols(table):
    op_cols = []
    ref_cols = []
    item_cols = []
    des_cols = []
    qty_cols = []
    for idx, _ in enumerate(table.columns):
        if 'op' in table.cell(OP_ROW, idx).text.lower():
            op_cols.append(idx)
        if 'ref.' in table.cell(HEADER_ROW, idx).text.lower():
            ref_cols.append(idx)
        if 'man.item no.' in table.cell(HEADER_ROW, idx).text.lower():
            item_cols.append(idx)
        if 'description' in table.cell(HEADER_ROW, idx).text.lower():
            des_cols.append(idx)
        if 'qty' in table.cell(HEADER_ROW, idx).text.lower():
            qty_cols.append(idx)
            
    right_header_cols = {'op':int, 'ref': int ,'item':int,'des':int,'qty':int}
    try:
        right_header_cols['op'] = 19
        right_header_cols['ref'] = ref_cols[2]
        for col in item_cols:
            if col > right_header_cols['ref']:
                right_header_cols['item'] = col
                break
        for col in des_cols:
            if col > right_header_cols['ref']:
                right_header_cols['des'] = col
                break
        for col in qty_cols:
            if col > right_header_cols['ref']:
                right_header_cols['qty'] = col
                break
    except:
        pass
    return right_header_cols

def GetLeftItemsList(table):
    item = []
    items = []
    headers = getLeftHeaderCols(table)
    for i, _ in enumerate(table.rows) : # left
        try:
            op = table.cell(OP_ROW, headers['op']).text
            ref = table.cell(FIRST_ROW + i, headers['ref']).text
            item = table.cell(FIRST_ROW + i, headers['item']).text
            des = table.cell(FIRST_ROW  + i, headers['des']).text
            qty = table.cell(FIRST_ROW + i, headers['qty']).text
        except:
            continue
        if qty.isnumeric():
            item = [op, ref, item, des, qty]
            items.append(item)
    return items

def GetRightItemsList(table):
    item = []
    items = []
    headers = getRightHeaderCols(table)
    for i, _ in enumerate(table.rows) : # right
        
        try:
            op = table.cell(OP_ROW, headers['op']).text
            ref = table.cell(FIRST_ROW + i, headers['ref']).text.replace('\n','')
            item = table.cell(FIRST_ROW + i, headers['item']).text
            des = table.cell(FIRST_ROW  + i, headers['des']).text
            qty = table.cell(FIRST_ROW + i, headers['qty']).text
        except:
            continue
        if qty.isnumeric():
            item = [op, ref, item, des, qty]
            items.append(item)
    return items

# GUI Functions

In [15]:
sopfile = ''

def open_file():
    global sopfile
    global file_name
    try:
        sopfile = filedialog.askopenfilename(initialdir=r'C:\Users\ParkGY\DocumentsCFLTYanadoo\DT Academy\SOPMeterial',title="select a file",
                                            filetypes =(("Word File","*.docx"),
                                            ("all files","*.*")))
    except:
        pass
    
def convert():
    doc = Document(sopfile)    
    df = pd.DataFrame(columns=['OperationStep','Ref.','Man.Item.No','Description','Qty'])
    for tb in doc.tables:
        if tb and len(tb.rows) > 10: # 최소 열줄은 넘겨야...
            lefts = GetLeftItemsList(tb)
            rights = GetRightItemsList(tb)
            for left in lefts:
                df.loc[len(df)] = left
            for right in rights:    
                df.loc[len(df)] = right
   
    f_name = sopfile.split('.')[0] + '.csv' 
    df.to_csv(f_name, encoding='utf-8-sig', index=False, mode='w', header=True)   


# GUI layout and Run

In [16]:
root = Tk()
root.title("Docx SOP Converter")
root.geometry("300x150")

# Frame 
frame = Frame(root)
frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_add_file = Button(frame, padx=5, pady=5, text="Open SOP", command=open_file)
btn_add_file.pack(fill="x",side="top", padx=5, pady=5)

btn_start = Button(frame, padx=5, pady=5, text="Convert", command=convert)
btn_start.pack(fill="x",side="top", padx=5, pady=5)
# Runs
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Program Files\Python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\ParkGY\AppData\Local\Temp\ipykernel_31308\3647710985.py", line 18, in convert
    lefts = GetLeftItemsList(tb)
  File "C:\Users\ParkGY\AppData\Local\Temp\ipykernel_31308\1804470304.py", line 63, in GetLeftItemsList
    headers = getLeftHeaderCols(table)
  File "C:\Users\ParkGY\AppData\Local\Temp\ipykernel_31308\1804470304.py", line 20, in getLeftHeaderCols
    return {'op':op_cols[0] ,'ref':ref_cols[0],'item':item_cols[0],'des':des_cols[0],'qty':qty_cols[0]}
IndexError: list index out of range
